In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
input_df = pd.read_excel('input.xlsx')

In [5]:
input_df.columns

Index(['Encounter_Date', 'Facility_Name', 'Appointment_Provider_Name',
       'Patient_Account_Number', 'Visit', 'Category', 'Patient_Name',
       'Financial_Class', 'Patient_Primary_Insurance_Name',
       'Patient_Secondary_Insurance_Name', 'CPT_Code', 'Modifier_1',
       'Modifier_2', 'Modifier_3', 'Visit_type', 'ICD_Code_1', 'ICD_Code_2',
       'ICD_Code_3', 'ICD_Code_4', 'ICD_Code_5', 'ICD_Code_6', 'ICD_Code_7',
       'ICD_Code_8', 'ICD_Code_9', 'Units', 'Arrived_Time', 'Departure_Time',
       'New_Patient', 'Encounter_Done_Status', 'Total_Time', 'Procedure_Notes',
       'Document_Section', 'GC_NGC', 'Status1', 'Status', 'Arrival_Status',
       'POS', 'INS_SelfPay', 'Visit_Notes', 'client'],
      dtype='object')

# Pat_dim

In [ ]:
pat_dim = pd.DataFrame(columns=['pat_dim_id','pat_source_key','pat_generated_name','pat_registered_on','pat_year_of_birth',
                                    'source_system',
                                    'created_date'])

pat_dim['pat_source_key'] = input_df['Patient_Account_Number']

pat_dim['pat_generated_name'] = input_df['Patient_Name']

pat_dim['pat_registered_on'] = input_df['Encounter_Date']

pat_dim.drop_duplicates(keep='first',inplace=True,ignore_index=True)

pat_id = []

for i in range(1,len(pat_dim)+1):
    id=f'pat_00{i}'
    pat_id.append(id)

pat_dim['pat_dim_id'] = pat_id

pat_dim['created_date'] = pd.to_datetime('today').date()

# Service_provider_dim

In [ ]:
Service_provider_dim=pd.DataFrame(columns=['service_provider_Id',
'service_provider_first_name',
'service_provider_last_name',
'service_provider_specialization',
'service_provider_job_role',
'service_provider_gender',
'service_provider_credential',
'service_provider_NPI',
'created_date'])

sp_name = input_df['Appointment_Provider_Name']

sp_name =  sp_name.str.split()

fn = []
ln = []
role = []

for i in sp_name:
    fn.append(i[-1])
    ln.append(i[0])
    role.append(i[1][0:-1])

Service_provider_dim['service_provider_first_name'] = fn
Service_provider_dim['service_provider_last_name'] = ln
Service_provider_dim['service_provider_job_role'] = role

Service_provider_dim.drop_duplicates(keep='first',inplace=True,ignore_index=True)

ser_id = []
for i in range(1,len(Service_provider_dim)+1):
    id = f'sp_00{i}'
    ser_id.append(id)

Service_provider_dim['service_provider_Id'] = ser_id

Service_provider_dim['created_date'] = pd.to_datetime('today').date()

# visit_type_dim

In [ ]:
visit_type_dim =pd.DataFrame(columns=['visit_type_dim_id',
'visit_type_dim_code',
'visit_type_dim_desc',
'created_date'])

visit_code=input_df['Visit_type'].drop_duplicates(keep='first',ignore_index=True)

visit_type_dim['visit_type_dim_code'] = visit_code

visit_desc={'P':'Private (Normal treatment)',
	'E': 'EPS (Screening)',
	'W': 'Worker comp',
	'M' : 'Misc  [only Self pay]'}


visit_type_dim['visit_type_dim_desc'] = visit_type_dim['visit_type_dim_code'].map(visit_desc)

visit_id=[]
for i in range(1,len(visit_type_dim)+1):
    i=str(i)
    if len(i)==1:
        id=f'V_00{i}'
        visit_id.append(id)
    elif len(i)==2:
        id=f'V_0{i}'
        visit_id.append(id)
    else:
        id=f'V_{i}'
        visit_id.append(id) 

visit_type_dim['visit_type_dim_id'] = visit_id

visit_type_dim['created_date'] = pd.to_datetime('today').date()



# calendar_dim

In [ ]:
calendar_dim=pd.DataFrame(columns=['cal_date_id',
'cal_date',
'cal_year',
'cal_month',
'cal_quarter',
'cal_week_of_year',
'cal_day_date',
'day_name',                                
'created_date'])

start_date= pd.to_datetime('today').date()
end_date='2026-12-31'

calendar_dim['cal_date'] = pd.date_range(start_date,end_date,freq='D')
calendar_dim['cal_year'] = calendar_dim['cal_date'].dt.year

calendar_dim['cal_month'] = calendar_dim['cal_date'].dt.month
calendar_dim['cal_quarter'] = calendar_dim['cal_date'].dt.quarter 
calendar_dim['cal_day_date'] =  calendar_dim['cal_date'].dt.day
calendar_dim['day_name'] = calendar_dim['cal_date'].dt.day_name()
calendar_dim['created_date'] = pd.to_datetime('today').date()

week = calendar_dim['cal_date'].dt.isocalendar().week-1
calendar_dim['cal_week_of_year'] = week

cl_id = []
for i in range(1,len(calendar_dim)+1):
    id = f'cal_date_00{i}'
    cl_id.append(id)

calendar_dim['cal_date_id'] = cl_id

# visit_category_dim

In [ ]:
visit_category_dim=pd.DataFrame(columns=['visit_category_dim_id',
'visit_category_code',
'visit_category_desc',
'created_date'])

visit_category_dim

visit_category_dim['visit_category_desc'] = input_df['Category']

visit_category_dim['created_date'] = pd.to_datetime('today').date()

visit_category_dim['visit_category_code'] = visit_category_dim['visit_category_desc'].apply(lambda x:x[0:3])

visit_category_dim.drop_duplicates(keep = 'first',inplace = True,ignore_index=True)

vc_id = []
for i in range(1,len(visit_category_dim) +1):
    id=f'VC_00{i}'
    vc_id.append(id)

visit_category_dim['visit_category_dim_id'] = vc_id

# contract_type_dim

In [ ]:
contract_type_dim=pd.DataFrame(columns=['contract_type_id',
'contract_type_code',
'contract_type_desc',
'created_date'])

contract_type_desc=input_df['Financial_Class'].drop_duplicates(keep='first',ignore_index=True)

contract_type_dim['contract_type_desc']=contract_type_desc
contract_type_dim['contract_type_code']=contract_type_dim['contract_type_desc'].apply(lambda x:x[0:3])

contract_type_dim['created_date'] = pd.to_datetime('today').date()

cc_id = []
for i in range(1,len(contract_type_dim) +1):
    id=f'C_00{i}'
    cc_id.append(id)

contract_type_dim['contract_type_id'] = cc_id

# insurance_firms_dim

In [ ]:
insurance_firms_dim=pd.DataFrame(columns=['ins_firm_id',
'ins_firm_code',
'ins_firm_name',
'created_date'])

ins_firm_name = input_df['Patient_Primary_Insurance_Name'].drop_duplicates(keep='first', ignore_index=True)

insurance_firms_dim['ins_firm_name'] = ins_firm_name

insurance_firms_dim['ins_firm_code']= insurance_firms_dim['ins_firm_name'].apply(lambda x:x[0:3]) 

insurance_firms_dim['created_date'] = pd.to_datetime('today').date()

if_id = []
for i in range(1,len(insurance_firms_dim) +1):
    id=f'INS_00{i}'
    if_id.append(id)

insurance_firms_dim['ins_firm_id'] = if_id

# financial_class_dim

In [ ]:
financial_class_dim=pd.DataFrame(columns=['financial_class_dim_id',
'financial_class_dim_code',
'financial_class_dim_desc',
'created_date'])

financial_class_dim_desc=input_df['Financial_Class'].drop_duplicates(keep='first',ignore_index=True)
financial_class_dim['financial_class_dim_desc'] = financial_class_dim_desc
financial_class_dim['created_date'] = pd.to_datetime('today').date()
financial_class_dim['financial_class_dim_code']=financial_class_dim['financial_class_dim_desc'].apply(lambda x:x[0:3])

fin_id = []
for i in range(1,len(financial_class_dim) +1):
    id=f'FC_00{i}'
    fin_id.append(id)

financial_class_dim['financial_class_dim_id'] = fin_id

# cpt_code_dim

In [ ]:
cpt_code_dim=pd.DataFrame(columns=['cpt_dim_id',
'cpt_code',
'cpt_desc',
'created_date'])

cpt_code = input_df['CPT_Code'].drop_duplicates(keep = 'first',ignore_index=True)
cpt_code_dim['cpt_code'] = cpt_code
cpt_code_dim['cpt_desc'] = cpt_code_dim['cpt_code'].apply(lambda x:str(x)[0:3]) 

fin_id = []
for i in range(1,len(cpt_code_dim) +1):
    id=f'CP_00{i}'
    fin_id.append(id)

cpt_code_dim['cpt_dim_id'] = fin_id

cpt_code_dim['created_date'] = pd.to_datetime('today').date()

# clinic_dim

In [ ]:
clinic_dim=pd.DataFrame(columns=['clinic_dim_id',
'clinic_source_id',
'clinic_name',
'clinic_type',
'clinic_size',
'clinic_operational_since',
'clinic_region',
'client_name',
'created_date'])

clinic_name=input_df['Facility_Name'].drop_duplicates(keep='first',ignore_index=True)
clinic_dim['clinic_name'] = clinic_name
clinic_dim['clinic_source_id']=clinic_dim['clinic_name'].apply(lambda x:str(x)[0:3]) 
clinic_dim['created_date'] = pd.to_datetime('today').date()

clinic_dim_id=[]
for i in range(1,len(clinic_name)+1):
    i=str(i)
    if len(i)==1:
        id=f'CL_00{i}'
        clinic_dim_id.append(id)
    elif len(i)==2:
        id=f'CL_0{i}'
        clinic_dim_id.append(id)
    else:
        id=f'CL_{i}'
        clinic_dim_id.append(id) 


clinic_dim['clinic_dim_id']=clinic_dim_id

In [231]:
with pd.ExcelWriter("dimensions.xlsx") as writer:
    pat_dim.to_excel(writer, sheet_name="Patient_dim", index=False)
    Service_provider_dim.to_excel(writer, sheet_name="Service_provider_dim", index=False)
    visit_type_dim.to_excel(writer, sheet_name="visit_type_dim", index=False)
    calendar_dim.to_excel(writer, sheet_name="calendar_dim", index=False)
    visit_category_dim.to_excel(writer, sheet_name="visit_category_dim", index=False)
    contract_type_dim.to_excel(writer, sheet_name="contract_type_dim", index=False)
    insurance_firms_dim.to_excel(writer, sheet_name="insurance_firms_dim", index=False)
    financial_class_dim.to_excel(writer, sheet_name="financial_class_dim", index=False)
    cpt_code_dim.to_excel(writer, sheet_name="cpt_code_dim", index=False)
    clinic_dim.to_excel(writer, sheet_name="clinic_dim", index=False)